In [8]:
# 서울시 응답소 - 시장에게 바란다 게시판 크롤링
# https://eungdapso.seoul.go.kr/

# Step 1. 필요한 모듈 및 라이브러리 설치

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import sys
import re
import math
import pandas as pd
import xlwt

import random
import os

In [27]:
# Step 2. 사용자에게 크롤링 정보 입력 받기

print("=" *80)
print("서울시 응답소 게시판 크롤링하기")
print("=" *80)

query_txt = '서울시응답소'

cnt = int(input('1. 크롤링 할 건 수는 몇 건 입니까?: '))
page_cnt = math.ceil(cnt / 10)

f_dir = input("2. 결과 파일을 저장할 폴더명만 쓰세요(예:c:\\temp\\):")

# 저장될 파일위치와 이름을 지정
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name = f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name = f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name = f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'

서울시 응답소 게시판 크롤링하기
1. 크롤링 할 건 수는 몇 건 입니까?: 30
2. 결과 파일을 저장할 폴더명만 쓰세요(예:c:\temp\):c:\temp\


In [29]:
# Step 3. 크롬 드라이버를 사용해서 웹 브라우저를 실행

s_time = time.time()

path = "C:\Temp\chromedriver_win32\chromedriver.exe"
driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get('https://eungdapso.seoul.go.kr/')
driver.maximize_window()

time.sleep(random.randrange(2, 5))

# Step 4. 시장에게 바란다 게시판 선택

driver.find_element_by_xpath('''/html/body/div[2]/div[1]/ul[1]/li[1]/a''').click()
time.sleep(3)

driver.find_element_by_xpath('''/html/body/div[2]/div[3]/div[1]/div/ul/li[4]/a''').click()



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\7138y\.wdm\drivers\chromedriver\win32\98.0.4758.102]
C:\Users\7138y\AppData\Local\Temp\ipykernel_11372\3805036746.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
C:\Users\7138y\AppData\Local\Temp\ipykernel_11372\3805036746.py:15: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('''/html/body/div[2]/div[1]/ul[1]/li[1]/a''').click()
C:\Users\7138y\AppData\Local\Temp\ipykernel_11372\3805036746.py:18: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=

In [14]:
# 타임스탬프
now1 = time.localtime()
s1 = '%04d-%02d-%02d-%02d-%02d-%02d' % (now1.tm_year, now1.tm_mon, now1.tm_mday, now1.tm_hour, now1.tm_min, now1.tm_sec)

print("\n")
page_cnt = math.ceil(cnt / 10)
print("크롤링 할 총 페이지 번호: ", page_cnt)
print("=" *80)

# Step 5. 전체 요약 정보 출력

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

result_list = soup.find('div', class_='pclist_table mt20').get_text( )
print(result_list)



크롤링 할 총 페이지 번호:  3



번호
제목
신청일
조회수


890
 서계동 구)주거환경개선지구를 재개발에...
2022-02-17
9


889
 차별없는 교육이 되어야하지 않겠습니까...
2022-02-17
7


888
 장애인의 이동권 예산을 의무적으로 보...
2022-02-17
143


887
 보육 발전을 위해 애써 주심에 감사합...
2022-02-11
56


886
 장애인 지하철 엘리베이터 설치
2022-02-09
113


885
 보육특별시-차액보육료 현실화/ 외국인...
2022-01-25
134


884
 시장님 외국인아동 보육료 지원은 공평...
2022-01-24
187


883
 서서울공원 활용 방안에 대한 건의 사...
2022-01-21
49


882
 사설 캠핑장 모닥불 행위로 인한 대기...
2022-01-21
97


881
 합리적이고 과학적인 방역지침을 만들어...
2022-01-16
184





In [32]:
# Step 6. 현재 창에서 열리는 페이지의 내용을 추출 후 돌아가기

no2 = [] # 게시글 번호
contents = [] # 게시글 내용
# hdate2 = [] # 신청일
# review2 = [] # 조회수

no = 1
click_cnt = 1

for x in range(1, page_cnt+1) :
    print("%s 페이지 내용 수집" %x)
    
    # 1 페이지에 있는 게시글 10건의 상세 내역 가져오기
    
    for i in range(2, 12) :
        driver.find_element_by_xpath("""/html/body/div[2]/div[3]/div[2]/div[2]/div/form/div[4]/table/tbody/tr[%s]/td[2]/a"""  %i).click()
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        content = soup.find('div', 'table_inner_desc').get_text()
        print(content)
        
        no2.append(no) # 게시글 번호에 추가
        contents.append(content.replace("\n","")+'\n') # 내용에 추가
        
        no += 1
        
        time.sleep(1)
        
        driver.back()
        
    time.sleep(2)
    
    if click_cnt > page_cnt+1 :
        break
        
    click_cnt += 1
    
    if (click_cnt % 5 == 0) :
        driver.find_element_by_link_text('''다음''').click()
    else : 
        driver.find_element_by_link_text("""%s""" %click_cnt).click()

1 페이지 내용 수집


C:\Users\7138y\AppData\Local\Temp\ipykernel_11372\2551909633.py:17: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath("""/html/body/div[2]/div[3]/div[2]/div[2]/div/form/div[4]/table/tbody/tr[%s]/td[2]/a"""  %i).click()



Q. 상담내용
                                    오세훈 시장님께서 추진하셨던 세운지구 개발화 사업이 박시장의 다시세운이라는 껍데기만 바꾸는 잘못된 사업추진 및 승인지연으로 인해 일부 지역만 누더기식 재정비가 되고 있습니다. 세운상가 외부를 제외한 대부분의 새운재정비지구는 이미 흉가와 같은 슬럼가로 변한 상황에서 안전 및 치안에 심각한 문제가 있습니다. 전 박시장의 임기중에 서울시에서 계속 개발에 대한 지연 결정 및 번복으로 인해 일부 지역은 해제를 일방적으로 통보를 했습니다.직주근접에 가장 적합한 도심내 지역을 주거가능 지역으로 개발하여 양적 확대가 필요한 상황에서 새운재정비 지구의 전반적인 재복원 및 개발이 시급하다고 제안드립니다. 오세훈 시장님의 추진력을 통해서 도심내 스럼가의 업그레이드 및 직주형 부동산 공급 확충을 위한 실행을 부탁드립니다.


Q. 상담내용
                                    오세훈 시장님, 우선 지난 선거에서 다시 시장으로 복직하신 것 축하드립니다. 그리고 많은 업무에 바쁘실 줄 압디다. 그러나 시간이 걸리더라도 개선해야 겠기에 간단히 적습니다.저는 00대학교에서 토양학을 전공으로 하는 000 교수입니다.지난 시장님 재임 시절 C40 세계도시정상회의 즈음해서 몇 번 건의한 내용입니다.지금은 그때보다 데 증요하고 심각하게 받아들여야 하겠기에 다시 의견을 묻습니다.기후변화와 이에 따른 불확도가 점점 커졌습니다.제가 그때 지적했던 것은 도시개발이 되면서 지표는 아스팔트로 덮히고 있는데, 심지어 사람이 다니는 도보도 숨쉬기 어렵게 되어가고 있습니다. 보기에만 좋은 녹색 폐타이어와 장식 등을 맨 위에 놓고 그 밑에 콘크리트를 쳐, 빗물도 하수도로 빠지게 되고, 땅이 숨도 제대로 쉬지 못하는 도시로 변하고 있습니다. 이래서 도시가 과연 기후변화 시대에 맞춰 지속성을 가질 수 있을까라는 의문이 계속 듭니다.탄소와 수자원의 흡수원인 토양이 제대로 숨을 쉬지 못하면, 쉽게 말해 가로수 등의

C:\Users\7138y\AppData\Local\Temp\ipykernel_11372\2551909633.py:44: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  driver.find_element_by_link_text("""%s""" %click_cnt).click()


2 페이지 내용 수집

Q. 상담내용
                                    서울시정 수행에 수고가 많으십니다. 권역별 시민청 조성 사업과 관련하여 질문 드립니다.2020년에 권역별 시민청 조성 사업에 대해 타당성 연구가 진행된 것으로 알고 있습니다. 문화 사각지대에 시민청이 건립된다는 뉴스를 접하고 서울시민으로서 기대가 컸습니다. 2020년 타당성 조사 후 현재 진행되고 있는 상황과 앞으로의 계획은 어떻게 되는지 궁금합니다. 감사합니다.
                                

Q. 상담내용
                                    한양도성안의 1)삼성그룹 현대그룹 등은 강남으로, 2)행정은 세종시로 떠나버렸습니다. 3)관광관련산업도 대부분 문을 닫았습니다. 길을 잃었습니다.도심 공동화 현상이 심각하고, 관련 자영업 소상공인의 폐업이 줄을 잇고 있으며, 일자리가 급감하고 있습니다.다행히 지역 전통 금융산업인 한국은행, 미래에셋, 하나외환은행 등과 다국적기업 서울사무소가 명맥을 유지하고 있습니다. 한편 홍콩은 해가 저물고 있어 금융기업 다국적기업 등이 다른 지역으로 이전하거나 준비하고 있습니다. 일부는 광화문을 선택합니다.런던 뉴욕 홍콩 싱가포르와 같이 광화문을 세계적인 금융허브와 다국적기업 비즈니스센터로 집중 육성할 절호의 기회입니다.광화문을 홍콩과 같이 세계적인 금융비즈니스센터로 육성하기 위해 시장님께서 직접 추진위원장을 맡아주시면 참 좋겠습니다.감사합니다.
                                

Q. 상담내용
                                    C19 극복을 위한 [안심업장] 인증 구조식당의 예를 들면 테이블별로 구역을 묶어 투명 아크릴로 분리하여 감염 발생이 있는 경우 그 구역만 폐쇄하고 감염과 무관한 구역은 계속 영업하기 위한 고안임1. 공간 ABC는 각각 밀폐된 독립공간이며 독립된 출입문이 있다.2. 각 객실에는 건물 외부에 연결된 


Q. 상담내용
                                    서울시장님저는 강북구 수유12구역에 살고 있는 주민입니다.본인은 이 지역에 평생살고자  하는 주민입니다.본인의 집에는 4가구의 독거노인들이 거주하고 있습니다,수유12구역은 갈곳이 없는  어려운 이웃들이 살기 좋은 곳입니다.몸이 편찮으신 분들.거동이 불편하신분들이 맘편히 살고 계십니다.느닷없는 공공주도3080의 발표로 이렇게 어려운 독거노인들은 어디로 쫓겨납니까더이상 갈곳이 없습니다.본인집에 거주하는 독거노인들은 공공주도3080으로 잠을 못이루고 매일 걱정과 불안에 병이 났습니다.이렇게 어려운 이웃들을 마음 편하게 살수 있도록 해주십시요,공공주도3080을 막아주십시요.수유12구역을 이대로 살수 있도록 해주십시요.
                                

Q. 상담내용
                                     장애인 복지카드를 가지고 있어도 수급권자가 아니면 의료비 지출이 많이 부담스럽습니다. 장애인 의료비 지원을 늘려주시기 바랍니다
                                

Q. 상담내용
                                    서울시 광장과 대학병원 주차장에 콘테이너 박스 설치해서 항체치료제 투약센타를 만들어 주십시오. 렉키로나 처방 권고에 해당하는 환자부터 무조건 렉키로나 투여하고 집에서 재택하도록 만드세요.병상부족, 확진자 증가세, 사망자 수 등을 크게 개선시킬 수 있습니다.그리고 요즘 국민들이 코로나 검사 받으려고 추운데 줄서서 기다리고 그러던데...... 검사하러 오신 분들 집에서 검사할 수 있게 자가진단키트 무료로 제공해주세요자가진단키드 받은 사람을 핸드폰 어풀 깔아서 등록하고 검사 결과를 핸드폰으로 반드시 입력하게 하세요. 집에서 검사한 결과가 양성으로 나오면 코로나 검사 받는 곳에서 한 번 더 검사해서 확진자로 판명되면 곧바로 치료할 수 있게 조치를 취하시면 좋을거 같습니다.검사 받는

In [33]:
seoul = pd.DataFrame()
seoul['번호'] = no2
seoul['내용'] = contents

# txt 형식의 파일로 저장
f = open(ff_name, 'a', encoding='UTF-8')
f.write(str(contents) + '\n')
f.close()

# csv 파일로 저장
seoul.to_csv(fc_name, encoding="utf-8-sig")

# 엑셀 형태로 저장
seoul.to_excel(fx_name)

e_time = time.time( )
t_time = e_time - s_time

print("\n")
print("=" *80)
print("총 소요시간은 %s 초 입니다" %round(t_time, 1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)

driver.close()



총 소요시간은 193.8 초 입니다
파일 저장 완료: txt 파일명 : c:\temp\2022-03-05-17-39-03-서울시응답소\2022-03-05-17-39-03-서울시응답소.txt 
파일 저장 완료: csv 파일명 : c:\temp\2022-03-05-17-39-03-서울시응답소\2022-03-05-17-39-03-서울시응답소.csv 
파일 저장 완료: xls 파일명 : c:\temp\2022-03-05-17-39-03-서울시응답소\2022-03-05-17-39-03-서울시응답소.xls 


C:\Users\7138y\AppData\Local\Temp\ipykernel_11372\399671364.py:14: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  seoul.to_excel(fx_name)
